# Maybe and Either

## Outline

* Missing values in Haskell

* The Maybe type

* The Either type

* A practical example

In this lesson, we will learn how to handle possible missing values in Haskell. We will introduce the types `Maybe` and `Either` and finish the lesson with some practical examples.

## Missing values in Haskell

Dealing with real life data in computer programs always opens up the question how to handle the data if some of it is missing. 

You have learned now how input output works in Haskell and imagine when reading a file from disk, some data you might expect is missing. 

Another example would be fetching data from a web server, e.g. a REST API and getting an error response. Of course a JSON string that the API returns can always contain an error description. 

But how to represent this in Haskell when you need to have an error type that also contains a string for the error description?

The answer to this is of course **parameterized types**. In many other programming languages a missing value is represented by the null value. 

The Haskell types that are used for this are called **Maybe** and **Either**. They are not just used by incentive of the programmer but are sometimes also returned by other Haskell functions. 

## The Maybe type

The Maybe type in Haskell is used when to represent data that is missing, but no other information is attached to the missing value as for example an error message. 

In Haskell the Maybe type allows you to write safer code because potential missing values are caught. 

The Maybe type structure is:
```haskell
data Maybe a = Nothing | Just a
```

Below are two variable declarations:

In [21]:
var1, var2 :: Maybe Int
var1 = Nothing
var2 = Just 7

var1
var2

Nothing

Just 7

If a value is missing the Maybe type returns a Nothing and if the value is present it returns for instance a Just 7. 

Below you can see an example of how Maybe values are used by the Map data structure that holds key and value pairs.

NOTE: We will learn more in depth about the **Data.Map** module in the next lesson.

In [3]:
import qualified Data.Map as Map

-- The Map functions we will use
:i Map.fromList
:i Map.lookup

values :: [Int]
values = [1,3,5]

strings :: [String]
strings = ["one","three","five"]

pairs :: [(Int, String)]
pairs = zip values strings

myMap :: Map.Map Int String
myMap = Map.fromList pairs

lookup1, lookup2 :: Maybe String
lookup1 = Map.lookup 1 myMap
lookup2 = Map.lookup 2 myMap

print lookup1
print lookup2

Just "one"

Nothing

Map.lookup :: Ord k => k -> Map.Map k a -> Maybe a 	-- Defined in ‘Data.Map.Internal’

Once you have a Map object you can also transform it back to a list with the function `toList`. You can also store your own defined object types in a Map. 

Another pair of useful functions are `isJust` and `isNothing` which check on a Maybe value wether it is of a Just type or a Nothing type and return a Bool.

In [5]:
import qualified Data.Map as Map
import Data.Maybe (isJust, isNothing)

data Numbers = One | Two deriving (Show)

simpleMap :: Map.Map Int Numbers
simpleMap = Map.fromList [(1,One),(2,Two)]

listFromMap :: [(Int, Numbers)]
listFromMap = Map.toList simpleMap

print $ head listFromMap
print $ isJust $ Map.lookup 1 simpleMap
print $ isJust $ Map.lookup 3 simpleMap

(1,One)

True

False

In the last example we show how we can combine some of the things we learned. We will iterate through a Map and filter out only the existing values. 

We pretend that we do not know in advance which are the existing values. Another function we will use here is `fromJust` that takes a Just value and extracts the contained value from it. 

In [22]:
import qualified Data.Map as Map
import Data.Maybe (isJust, fromJust)

boxes :: Map.Map Int String
boxes = Map.fromList [(1,"toy"),(3,"watch"),(5,"book")]

boxesToCheck = [1..5]

getBoxContents :: [Int] -> Map.Map Int String -> [String]
getBoxContents ids catalog = map fromJust $ filter isJust $ map getContents ids
 where getContents = \id -> Map.lookup id boxes
 
print $ getBoxContents boxesToCheck boxes

Line 10: Use catMaybes
Found:
map fromJust $ filter isJust $ map getContents ids
Why not:
Data.Maybe.catMaybes (map getContents ids)Line 11: Avoid lambda using `infix`
Found:
\ id -> Map.lookup id boxes
Why not:
(`Map.lookup` boxes)

["toy","watch","book"]

## The Either type
The Either type in Haskell is used when to represent data that is missing and we want to return another value, as for example an error message. 

This is used instead of Maybe for instance when the reasons for a missing value are important and you want to share it with the user. 

Another possibility would be that you want to return a default value when the real one is missing.

This is the Either type structure:
```haskell
data Either a b = Left a | Right b
```

Below you can see two variable declarations. For missing values we ussually use the Left data constructor.

In [23]:
var1, var2 :: Either String Int
var1 = Left "Error: Value is missing."
var2 = Right 7

var1
var2

Left "Error: Value is missing."

Right 7

Lets look at an example where the user has to input a number up to 100 and that is checked weather it is prime. 

We now have the option to return an error message that explains why a certain number is not a valid input.

In [24]:
-- Function for generating prime numbers
primeNumbers :: Int -> [Int]
primeNumbers n = getPrimes [2..]
  where getPrimes list@(x:xs) = if x > n 
                                then [] 
                                else filterPrimes list
        filterPrimes (x:xs) = x : getPrimes (filter ((/= 0) . (`mod` x)) xs)

-- Function that checks if a number is valid and prime
isNumberPrime :: Int -> Either String Bool
isNumberPrime n
  | n < 2 = Left "Prime numbers are larger than 1."
  | n > 100 = Left "Number is to large for testing."
  | otherwise = Right (n `elem` primeNumbers n)

main :: IO ()
main = do
  print "Enter a number for testing up to 100:"
  input <- getLine
  let p = read input
      result = isNumberPrime p
  print result

main

"Enter a number for testing up to 100:"
Right True

An example of a build-in Haskell function that returns a Either type is `eitherDecode` that comes with the **Aeson** library. 

It is used to parse a JSON file and returns either an object of type a, or a string that contains an error message of the parsing. 

The type signature of the function you can see below. You will learn about the Aeson module in lesson 18.
```haskell
eitherDecode :: FromJSON a => ByteString -> Either String a
```

## A practical example

Lets look now at an example where we define a type for handling errors on our own. We will call it `Path` and the constructors will be `Success` and `Failure`.

We will write a program that askes the user to provide some data to create a user account. 

The program performs some simple checks on the data and lets the user know weather the data passed the checks or not. 
 
The concept of checks follows the paradigm of Railway oriented programming where your checks can put you on a success or failure track. 

In [27]:

-- Here we define our error type and make it an instance of Show
data Path a b = Success a | Failure b
type StringPath = Path [String] String

instance Show b => Show (Path a b) where
  show (Success _) = "The data has passed all the checks."
  show (Failure msg) = show msg

-- The main function of the programm
main :: IO ()
main = do
      name <- getInput "What is your name?"
      password <- getInput "What is your password?"
      answer <- getInput "In which year where you born?"
      
      let userData = Success [name, password, answer] :: Path [String] String
      let result = checkYear . checkPassword . checkName $ userData
      print result

-- Helper functions
getInput :: String -> IO String
getInput question = do
      print question
      getLine

process :: StringPath -> (String -> StringPath) -> Int -> StringPath
process userData checkFunction index = 
  case userData of
      Success xs -> checkFunction $ xs !! index
      Failure msg -> Failure msg

-- Functions that make a certain check for the user data
checkName :: StringPath -> StringPath
checkName userData = process userData check 0
  where check input = if length (words input) > 1
                      then userData
                      else Failure "Name has to contain at least 2 parts."

checkPassword :: StringPath -> StringPath
checkPassword userData = process userData check 1
  where check input = if length input > 5
                      then userData
                      else Failure "Password has less than 6 characters."

checkYear :: StringPath -> StringPath
checkYear userData = process userData check 2
  where check input = if read input > 1900 && read input < 2020 
                      then userData
                      else Failure "Birth year has to be between 1900 and 2020."
                      
main

"What is your name?"
"What is your password?"
"In which year where you born?"
The data has passed all the checks.

## Recap

In this lecture, we have introduced the handling of missing data and errors in Haskell. 

- Haskell has two types for missing data called Maybe and Either. 
- Maybe can be used when the reason for the missing data is not important.
- Either can be used when the reason for the missing data is important or when you want to return a default value.
- You can also define your own type for handling errors and can use Railway oriented programming for error handling.